In [1]:
from itertools import combinations
from typing import List, Tuple

import numpy as np
from tqdm.notebook import tqdm

In [2]:
def element_comb_val(comb: Tuple[int], i: int, j: int, N: int, D: int, betas_opt: np.ndarray, betas: np.ndarray, var: np.ndarray) -> float:
    res = 1.
    for idx in comb:
        res *= var[idx]
    
    not_comb_j = [idx for idx in range(N) if idx not in comb and idx != j]
    
    LHS = np.concatenate([betas_opt[i, :].reshape(1, D), betas[not_comb_j, :]])
    RHS = np.concatenate([betas[j, :].reshape(1, D), betas[not_comb_j, :]])
    
    #print(f"comb: {comb}", i, j, LHS.shape, RHS.shape, f"comb_i: {i},{not_comb_j}", f"comb_j: {j},{not_comb_j}")
    
    return res * np.linalg.det(LHS @ RHS.T)
    

def element_wise_val(S: List[int], i: int, j: int, N: int, D: int, betas_opt: np.ndarray, betas: np.ndarray, var: np.ndarray) -> float:
    sum = 0.
    for k in range(len(S) + 1):
        for comb in combinations(S, k):
            sum += element_comb_val(comb, i, j, N, D, betas_opt, betas, var)
    return sum.item()
    

def verify_conjecture_1() -> None:
    N = np.random.randint(2, 10)
    D = np.random.randint(1, 100)
    betas_opt = np.random.random(size=(N, D))
    betas = betas_opt + np.random.random(size=(N, D))
    var = np.random.random(size=(N, 1))
    
    K = betas_opt @ betas.T
    C = betas @ betas.T
    V = np.eye(N) * var
    W = K @ np.linalg.inv(C + V)
    
    W_element_wise = np.empty(shape=(N, N))
    det = 0.
    det_S = list(range(N))
    for k in range(len(det_S) + 1):
        for comb in combinations(det_S, k):
            res = 1.
            for idx in comb:
                res *= var[idx]
                
            not_comb = [idx for idx in range(N) if idx not in comb]
            det += (res * np.linalg.det(betas[not_comb, :] @ betas[not_comb, :].T))
    det = det.item()
    
    for i in range(N):
        for j in range(N):
            S = [idx for idx in range(N) if idx != j]
            W_element_wise[i, j] = element_wise_val(S, i, j, N, D, betas_opt, betas, var)
    
    W_element_wise /= det
    
    assert np.allclose(W, W_element_wise), f"N: {N}, D: {D}, W_element_wise: {W_element_wise}, W: {W}"

In [3]:
for _ in tqdm(range(1000000)):
    verify_conjecture_1()

  0%|          | 0/1000000 [00:00<?, ?it/s]

KeyboardInterrupt: 